In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

from ai.TrashClassifier import TrashClassifier
from ai.DataLoader2 import DataLoader
from env import *

EPOCHS = 100
ETA = 1e-2

In [2]:
model = TrashClassifier()
model = model.cuda()

In [3]:
train_dloader = DataLoader(TRAIN_DATA_PATH, TRASH_CAT)
valid_dloader = DataLoader(VALID_DATA_PATH, TRASH_CAT)

Number of data batch: 540
Number of batch: 8
Number of data batch: 80
Number of batch: 1


In [4]:
optimizer = optim.Adam(model.parameters(), lr=ETA)
criterion = nn.NLLLoss()

In [5]:
for e in range(EPOCHS):
    
    train_loss = 0.0
    train_acc = 0.0
    valid_loss = 0.0
    valid_acc = 0.0
    
    for x_batch, y_batch in train_dloader.next_batch():
        x_batch = torch.FloatTensor(x_batch).cuda()
        y_batch = torch.LongTensor(y_batch).cuda()
        
        logps = model(x_batch)
        loss = criterion(logps, y_batch)
        
        with torch.no_grad():
            train_loss += loss.item()
            ps = torch.exp(logps)
            
            ps_k, cls_k = ps.topk(1, dim=1)
            equal = cls_k == y_batch.view(*cls_k.size())
            train_acc += torch.mean(equal.type(torch.FloatTensor))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (e+1) % 10 == 0:
        with torch.no_grad():
            model.eval()

            for x_batch, y_batch in valid_dloader.next_batch():
                x_batch = torch.FloatTensor(x_batch).cuda()
                y_batch = torch.LongTensor(y_batch).cuda()

                logps = model(x_batch)
                loss = criterion(logps, y_batch)

                valid_loss += loss.item()

                ps = torch.exp(logps)
                ps_k, cls_k = ps.topk(1, dim=1)
                equal = cls_k == y_batch.view(*cls_k.size())
                valid_acc += torch.mean(equal.type(torch.FloatTensor))

            train_loss /= len(train_dloader)
            train_acc /= len(train_dloader)
            valid_loss /= len(valid_dloader)
            valid_acc /= len(valid_dloader)

            print(f"Epochs: {e+1}/{EPOCHS}")
            print(f"Train loss: {train_loss:.8f}")
            print(f"Train acc: {train_acc:.8f}")
            print(f"Valid loss: {valid_loss:.8f}")
            print(f"Valid acc: {valid_acc:.8f}")

            model.train()

Epochs: 10/100
Train loss: 1.04215227
Train acc: 0.53125000
Valid loss: 1.74280286
Valid acc: 0.43750000
Epochs: 20/100
Train loss: 0.93946547
Train acc: 0.60546875
Valid loss: 14.52096367
Valid acc: 0.34375000
Epochs: 30/100
Train loss: 0.70602417
Train acc: 0.73437500
Valid loss: 9.68461609
Valid acc: 0.34375000
Epochs: 40/100
Train loss: 0.40707511
Train acc: 0.87500000
Valid loss: 2.14607334
Valid acc: 0.40625000


KeyboardInterrupt: 

In [ ]:
model.save(CLF_CKPT_PATH)